In [ ]:
import numpy as np
from scipy.optimize import differential_evolution
from scipy.optimize import basinhopping

Network class definition

In [ ]:
class ACT_reservoir:
    def __init__(self, input_dim, hidden_dim, output_dim, max_iter=20):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.max_iter = max_iter
        
        self.total_dim = 1 + input_dim + hidden_dim + output_dim + 1  # Including bias and halting node
        self.W = np.random.uniform(-2, 2, (self.total_dim, self.total_dim))

        # Index ranges for different parts of the state vector
        self.input_indices = range(1, input_dim + 1)
        self.hidden_indices = range(input_dim + 1, input_dim + hidden_dim + 1)
        self.output_indices = range(input_dim + hidden_dim + 1, input_dim + hidden_dim + output_dim + 1)
        self.halting_index = input_dim + hidden_dim + output_dim + 1

    def relu(self,x):
        return np.maximum(0, x)

    def predict(self, xi):
        # Create initial state vector with given input, xi, and set bias unit to 1
        self.x = np.zeros((self.total_dim, 1))
        self.x[0] = 1  # Bias unit
        self.x[self.input_indices] = xi.reshape((self.input_dim, 1))  # Input part

        t = 0
        while t < self.max_iter and self.x[self.halting_index][0] <= 1:
            self.x = self.relu(self.W @ self.x)
            t += 1

        # Extracting and returning the output part of the state vector
        return self.x[self.output_indices],t

    def summary(self):
        print("Total Parameters:", self.total_dim ** 2)  # Since W is a square matrix
        print("Input Dimension:", self.input_dim)
        print("Hidden Dimension:", self.hidden_dim)
        print("Output Dimension:", self.output_dim)
        print("Max Iterations:", self.max_iter)

Example Usage

In [ ]:
# Example usage
model = ACT_reservoir(2, 10, 1) 
model.summary()

In [ ]:
# Predict with a sample input
xi = np.array([0.5, 0.7])
output,steps = model.predict(xi)
print("Output:", output)
print("Steps:",steps)

Generating data

In [ ]:
# Number of samples
num_samples = 1000

# Generate random pairs of integers between 0 and 100
factors = np.random.randint(0, 101, size=(num_samples, 2))

# Compute the product of each pair
products = np.prod(factors, axis=1).reshape(-1, 1, 1)

# Combine factors and products into a single dataset
# Each element in the dataset will be ([factor1, factor2], [[product]])
dataset = [(factors[i].reshape(-1, 1), products[i]) for i in range(num_samples)]

In [ ]:
dataset

Defining fitness function

In [ ]:
def fitness(weights, model, train_data, batch_size=5):
    # Reshape the weights and set them in the model
    model.W = weights.reshape(model.W.shape)

    # Randomly sample a batch from the training data
    batch_indices = np.random.choice(len(train_data), batch_size, replace=False)
    batch = [train_data[i] for i in batch_indices]

    total_loss = 0
    for xi, yi in batch:
        predicted_output, _ = model.predict(xi)
        # Assuming a simple mean squared error for the loss
        total_loss += np.mean((predicted_output.flatten() - yi) ** 2)

    return total_loss / len(batch) * 1e-3

In [ ]:
model = ACT_reservoir(2, 7, 1)
model.summary()

Differential Evolution

In [ ]:
# Bounds for the weights in the optimization
bounds = [(-4, 4)] * (model.total_dim ** 2)

# Use Differential Evolution with the new fitness function
result = differential_evolution(fitness, bounds, args=(model, dataset, 100), maxiter=100, disp=True,popsize=100,strategy="randtobest1bin")

# Set the optimized weights in the model
optimized_weights = result.x
model.W = optimized_weights.reshape(model.W.shape)

Simulated Annealing

In [ ]:
# Initialize your model
model = ACT_reservoir(input_dim=2, hidden_dim=10, output_dim=1, max_iter=20)

# Flatten the initial weights of the model
initial_weights = model.W.flatten()

# Bounds for the weights in the optimization
bounds = [(-2, 2)] * len(initial_weights)

# Define a minimizer_kwargs dictionary for basinhopping
minimizer_kwargs = {"method": "L-BFGS-B", "bounds": bounds, "args": (model, dataset)}

# Use Simulated Annealing to optimize the weights
result = basinhopping(fitness, initial_weights, minimizer_kwargs=minimizer_kwargs, niter=100, disp=True)

# Set the optimized weights in the model
optimized_weights = result.x
model.W = optimized_weights.reshape(model.W.shape)

TESTING/EVALUATION

In [ ]:
# Predict with a sample input
xi = np.array([1,1])
output,steps = model.predict(xi)
print("Output:", output)
print("Steps:",steps)